##### Copyright 2025 Denis Pasichny I.D: 305868283.[LinkedIn](https://www.linkedin.com/in/denis-pasichny-758637171/)

## 📖 Get started with Kaggle Notebooks

If this is your first time using Kaggle Notebooks, welcome! You can learn more about using Kaggle Notebooks [in the documentation](https://www.kaggle.com/docs/notebooks).

Here's how to get started:

**1. Verify Your Account (Required)**

To use the Kaggle Notebooks in this course, you'll need to verify your account with a phone number.

You can do this in your [Kaggle settings](https://www.kaggle.com/settings).

**2. Make Your Own Copy**

To run any code in this notebook, you first need your own editable copy.

Click the `Copy and Edit` button in the top-right corner.

![Copy and Edit button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

This creates a private copy of the notebook just for you.

**3. Run Code Cells**

Once you have your copy, you can run code.

Click the ▶️ Run button next to any code cell to execute it.

![Run cell button](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

Run the cells in order from top to bottom.

**4. If You Get Stuck**

To restart: Select `Factory reset` from the `Run` menu.

For help: Ask questions on the [Kaggle Discord](https://discord.com/invite/kaggle) server.

🚨 **NOTE:**  When you first start the notebook via running a cell you might see a banner in the notebook header that reads **"Waiting for the next available notebook"**. The queue should drop rapidly; however, during peak bursts you might have to wait a few minutes.

🚨 **NOTE:** Avoid using the **Run all** cells command as this can trigger a QPM limit resulting in 429 errors when calling the backing model. Suggested flow is to run each cell in order - one at a time. [See FAQ on 429 errors for more information.](https://www.kaggle.com/code/kaggle5daysofai/day-0-troubleshooting-and-faqs)

## ⚙️ Section 1: Setup

### Install dependencies

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk
```

### 1.1 Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to complete authentication.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


### 1.2 Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [2]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.3: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [38]:
retry_config = types.HttpRetryOptions(
    attempts=10,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)
print("✅ Configurate retry options completed successfully.")

✅ Configurate retry options completed successfully.


## ⚙️ Section 2: Build In Tools

### Create Severity Calculation function
This will the main tool when it comes to detection of disease

In [44]:
def calc_infection_severity(tmp: int, hmt: int, rh: int = 60) -> dict:
    '''
    Calculate severity of disease and return value between [0-5].

    Arguments:
    1. tmp - Measured temprature.
    2. hmt - Measured Humidity levels.
    3. rh - Measured relative humidity on the leaf.

    Returns:
    Dictionary with status and disease severity information.
    Success: {"status": "success", "severity_index": 4}
    Error: {"status": "error", "error_message": "Something went wrong while trying to calculate disease severity index"}
    '''
    if tmp < 0 or hmt < 0 or rh < 0:
        return {
            "status" : "error",
            "error_message" : "One or more of the parameters is invalid"
        }
    
    dsi=0
    if tmp > 5 and tmp < 28.5:
        if tmp > 11.5 and tmp < 15.5:
            dsi+=2
        else:
            dsi+=1

    if hmt > 70:
        dsi+=1

    if rh >= 60 and rh <= 80:
        dsi+=1
    elif rh > 80:
        dsi+=2

    return {
        "status" : "success",
        "severity_index" : dsi
    }

print("✅ Built-In Disease Severity tool created successfully.")

✅ Built-In Disease Severity tool created successfully.


## ⚙️ Section 3: Defining Agents and Weather MCP Server

### 3.1: Defining MCP for my Weather Agent

The workflow is simple:

1. Choose an MCP Server and tool
2. Create the MCP Toolset (configure connection)
3. Add it to your agent
4. Run and test the agent

**Step 1: Choose MCP Server**

We'll use the **[MCP Weather Server](https://github.com/adhikasp/mcp-weather)**

It provides hourly weather forecasts using the AccuWeather API.

In [14]:
#But before we set the MCP tool, let's get some valuable packages imported
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ MCP components imported successfully.")

✅ MCP components imported successfully.


**Step 2: Create the MCP Toolset**

The `McpToolset` is used to integrate an ADK Agent with an MCP Server.

**What the code does:**
- Uses `uvx` (In the context of the Model Context Protocol (MCP), `uvx` stands for uv tool execute. It is a command-line tool provided by the `uv` package manager that executes a Python package's command, much like npx does for Node.js packages. It automatically creates an isolated environment with the necessary dependencies for the command to run. ) to run the MCP server
- Connects to `mcp-weather.git`
- Filters to only use the `mcp-weather` tool (the server has others, but we only need this one)
- Note: You MUST first register at [Accu Weather API](https://developer.accuweather.com/home) to have a proper key (Please don't use mine :))

In [42]:
# MCP integration with MCP Weather Server
mcp_weather_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="uvx",  # Run MCP server via npx
            args=[
                "--from",
                "git+https://github.com/adhikasp/mcp-weather.git",
            ],
            env = {
                "ACCUWEATHER_API_KEY": "zpka_998111a8592b4570a24fca2ef101bf9e_b63a0e0b"
            },
            tool_filter=["mcp-weather"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


In [45]:
# Define a temerature conversion capabilities
tmp_converter_agent = LlmAgent(
    name="tmp_converter_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

print("✅ Temperature converter Agent created!")

✅ Temperature converter Agent created!


**Step 3: Add MCP tool to agent**

Let's add the `mcp_weather_server` to the agent's tool array and update the agent's instructions to handle requests to get forcast from Accu Weather API.

In [46]:
# Agent responsible for getting enviromental parameters for weather service
weather_agent = LlmAgent(
    name="weather_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your only task is to use the `mcp_weather_server` tool and retrieve the values of "temperature" and "relative_humidity".
    If "unit" value is not "C",then you MUST call `tmp_converter_agent` in order to convert the value of "temperature" to Celsius units.
    Your are STRICTLY forbidden from trying to execute the conversion yourself.
    """,
    tools=[mcp_weather_server,AgentTool(tmp_converter_agent)],
)
print("✅ Weather Agent created!")

✅ Weather Agent created!


In [40]:
# Agent who is tasked with executing distribution of tasks depending on the nature of usr's request
task_master_agent = LlmAgent(
    name="task_master_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a task master agent whose sole purpose is to execute a task in the following way:
    1.Read the request and try to parse out of the text the values of `temperature`, `humidity` and `rh`.
    If you were unsuccessful in getting `temperature` and `humidity` values from the request, ONLY THEN you call
    `weather_agent` to provide you with values of `temperature`, `humidity` and `rh`.
    2. You can ONLY call `calc_infection_severity` if you have values of atleast `temperature` and `humidity`.
    3. If User specificially asks to get values of `temperature` and `humidity`, return them.
    4. If User specificially asks to get value of severity of disease, return this value.
    """,
    output_key="severity_index",
    tools=[AgentTool(weather_agent),FunctionTool(calc_infection_severity)],
)
print("✅ task Agent created!")

✅ task Agent created!


In [47]:
# Create assert agent who validates the request before delegating the task to the task agent. Also in charge of providing the user with thourghl answer
assert_agent = LlmAgent(
    name="assert_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a polite and gental assistant.
    your task is to recieve a request from the user and try to answer as best as you can.

    When the user asks you anything, you do the following sequence of operations:
    1. You validate the user request is related to Basil Downey Mildew(BDM) disease.
    If user's request is NOT about BDM, kindly explain to him you can only assist him with questions concerning BDM disease.
    2. Use `task_master_agent` tool to process the users request. You are STRICTLY prohibited from answering the user's request yourself.
    3. After recieving final answer from `task_master_agent`, you should present the answer to the user in an understandable way.
    """,
    tools=[AgentTool(task_master_agent)],
)

print("✅ Assert Agent created!")

✅ Assert Agent created!


## ⚙️ Section 4: Getting the system up and running

### 4.1 Create the runner:

In [35]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=assert_agent)

### 4.2 Let's test our new system!

In [37]:
#Simple case when data is provided
response = await runner.run_debug("I have noticed that my Basil field has signs of BDM. What is the severity of the disease if the current tempreture is 21 degrees Celsius, humidity is around 70% and relative leave humidity(rh) is at 90%")


 ### Continue session: debug_session_id

User > I have noticed that my Basil field has signs of BDM. What is the severity of the disease if the current tempreture is 21 degrees Celsius, humidity is around 70% and relative leave humidity(rh) is at 90%


assert_agent > The severity index for Basil Downey Mildew (BDM) is 3.


In [39]:
# Another simple case in which we ask about topic unrelated to BDM
response = await runner.run_debug("Can you explain to me how to cook Spanish Salmon?")


 ### Continue session: debug_session_id

User > Can you explain to me how to cook Spanish Salmon?
assert_agent > I can only assist you with questions concerning Basil Downey Mildew (BDM) disease.


In [43]:
# Here is a request involving weather mcp
response = await runner.run_debug("I am a Basil farmer from North of Israel. I wasn't able to measure my field's temperature and humidity because of heavy rain last night. Could you help me get the temperature and humidity levels in northren Israel for the past 12 hours?")


 ### Continue session: debug_session_id

User > I am a Basil farmer from North of Israel. I wasn't able to measure my field's temperature and humidity because of heavy rain last night. Could you help me get the temperature and humidity levels in northren Israel for the past 12 hours?


ERROR:mcp.client.stdio:Failed to parse JSONRPC message from server
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/mcp/client/stdio/__init__.py", line 155, in stdout_reader
    message = types.JSONRPCMessage.model_validate_json(line)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pydantic/main.py", line 766, in model_validate_json
    return cls.__pydantic_validator__.validate_json(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pydantic_core._pydantic_core.ValidationError: 1 validation error for JSONRPCMessage
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='Provide a command to run with `uvx <command>`.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid
ERROR:mcp.client.stdio:Failed to parse JSONRPC message from server
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/

McpError: Connection closed

#### OOPS.. Had some server connection issues with the last example.
#### Well this is but a sample of a much greater scope of work I am about to be doing here ;)